Even though baseline bbox predictions are provided by the host, you may want to train your helmet detector. To do that it's useful to convert this competition csv file to COCO format, it's de facto standard format in object detection.

This code assume image dataset is created by this script.  
https://www.kaggle.com/bamps53/create-image-dataset

And also the dataset is provided here.  
https://www.kaggle.com/bamps53/nfl2021-train-images

In [1]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
tqdm.pandas()

class NumpyEncoder(json.JSONEncoder):
    """ 
    https://stackoverflow.com/questions/26646362/numpy-array-is-not-json-serializable
    Special json encoder for numpy types
    """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

class COCOConverter:
    """Class to convert competition csv to coco format."""
    def __init__(
        self,
        df: pd.DataFrame, 
        image_height: int = 720, 
        image_width: int = 1280, 
        type_agnostic: bool = False):
        
        self.image_height = image_height
        self.image_width = image_width
        self.type_agnostic = type_agnostic
        if self.type_agnostic:
            self.categories = [{"id": 1, "name": "Helmet"}]
        else:
            self.categories = [
                {"id": 1, "name": "impact_None",},
                {"id": 2, "name": "impact_Helmet"},
                {"id": 3, "name": "impact_Shoulder",},
                {"id": 4, "name": "impact_Body"},
                {"id": 5, "name": "impact_Ground",},
                {"id": 6, "name": "impact_Hand"},
            ]         
        self.df = self._initialize(df)

    def _get_file_name(self, row: pd.Series):
        base_name = row.video[:-4]
        file_name = f'{base_name}_frame{row.frame:04}.jpg'
        return file_name

    def _get_bbox(self, row: pd.Series):
        return [row.left, row.top, row.width, row.height]

    def _initialize(self, df: pd.DataFrame):
        # set category id
        if self.type_agnostic:
            df['impactType'] = 'Helmet'
            df['category_id'] = 1
        else:
            df['category_id'] = df['impactType'].map(
                {
                    'None': 1,
                    'Helmet': 2,
                    'Shoulder': 3,
                    'Body': 4,
                    'Ground': 5,
                    'Hand': 6
                }
            )
        # some preprocesses
        df['file_name'] = df[['video', 'frame']].progress_apply(self._get_file_name, axis=1)
        df['area'] = df['width'] * df['height']
        df['bbox'] = df[['left', 'top', 'height', 'width']].progress_apply(self._get_bbox, axis=1)
        df['iscrowd'] = 0
        return df
        

    def save(self, save_path):
        """
        Save as coco json format.
        But also has many supplemental items like gameKey or view.
        """
        df = self.df.copy()
        image_df = df[['gameKey', 'playID', 'view', 'video', 'frame', 'file_name']].drop_duplicates()
        image_df['height'] = self.image_height
        image_df['width'] = self.image_width
        
        # add image id to images. Note that it's called just "id".
        image_df['id'] = range(1, len(image_df) + 1)
    
        # add image id to annotations.
        df['image_id'] = df[['file_name']].merge(image_df[['file_name', 'id']])['id'].values
        df['id'] = range(1, len(df) + 1)

        print('start dumping...')
        coco_annotations = dict()
        coco_annotations['categories'] = self.categories
        coco_annotations['images'] = [dict(row) for _, row in image_df.iterrows()]
        coco_annotations['annotations'] = [dict(row) for _, row in df.iterrows()]
        json.dump(coco_annotations, open(save_path, 'w'), indent=4, cls=NumpyEncoder)

In [2]:
df = pd.read_csv('../input/nfl-health-and-safety-helmet-assignment/train_labels.csv')

In [3]:
play_ids = df['playID'].unique()
num_train = int(len(play_ids) * 0.8)
train_play_ids = df['playID'].unique()[:num_train]
valid_play_ids = df['playID'].unique()[num_train:]
print('number of train videos:', len(train_play_ids))
print('number of valid videos:', len(valid_play_ids))

In [4]:
train_df = df.query('playID in @train_play_ids').reset_index(drop=True).copy()
valid_df = df.query('playID in @valid_play_ids').reset_index(drop=True).copy()

print('number of train annotations:', len(train_df))
print('number of valid annotations:', len(valid_df))

In [5]:
%%time
train_coco = COCOConverter(train_df, type_agnostic=True)
train_coco.save('train.json')

In [6]:
%%time
valid_coco = COCOConverter(valid_df, type_agnostic=True)
valid_coco.save('valid.json')